# Courbe de lumière de Sagittarius A*

On veut utiliser la nouvelle classe AnalysisConfig

In [30]:
%matplotlib inline
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.coordinates import SkyCoord
import logging

from astropy.time import Time

log = logging.getLogger(__name__)

from astropy.coordinates import Angle

In [33]:
from gammapy.data import DataStore

from gammapy.modeling.models import PowerLawSpectralModel
from gammapy.modeling.models import PointSpatialModel
from gammapy.modeling.models import SkyModel, SkyModels
from gammapy.time import LightCurveEstimator
from gammapy.analysis import Analysis, AnalysisConfig

In [51]:
t2004 = Time(['2004-01-01T00:00:00', '2004-12-31T23:59:59'])
t2005 = Time(['2005-01-01T00:00:00', '2005-12-31T23:59:59'])
t2006 = Time(['2006-01-01T00:00:00', '2006-12-31T23:59:59'])
t2007 = Time(['2007-01-01T00:00:00', '2007-12-31T23:59:59'])
t2008 = Time(['2008-01-01T00:00:00', '2008-12-31T23:59:59'])
t2009 = Time(['2009-01-01T00:00:00', '2009-12-31T23:59:59'])
t2010 = Time(['2010-01-01T00:00:00', '2010-12-31T23:59:59'])
t2011 = Time(['2011-01-01T00:00:00', '2011-12-31T23:59:59'])
t2012 = Time(['2012-01-01T00:00:00', '2012-12-31T23:59:59'])
t2013 = Time(['2013-01-01T00:00:00', '2013-12-31T23:59:59'])
t2014 = Time(['2014-01-01T00:00:00', '2014-12-31T23:59:59'])
t2015 = Time(['2015-01-01T00:00:00', '2015-12-31T23:59:59'])
t2016 = Time(['2016-01-01T00:00:00', '2016-12-31T23:59:59'])
t2017 = Time(['2017-01-01T00:00:00', '2017-12-31T23:59:59'])
t2018 = Time(['2018-01-01T00:00:00', '2018-12-31T23:59:59'])
t2019 = Time(['2019-01-01T00:00:00', '2019-12-31T23:59:59'])

time_intervals = [t2004, t2005, t2006, t2007, t2008, t2009, t2010, t2011, t2012, t2013, t2014, t2015, t2016, t2017, t2018, t2019]

In [60]:
# Define which data to use and print some information
data_store = DataStore.from_dir("../../data/ash_stereo_Prod17_Calib0834_thsq64")
#data_store.obs_table.meta.update(dict(TIME_FORMAT= "absolute"))
data_store.info()

Data store:
HDU index table:
BASE_DIR: ../data/ash_stereo_Prod17_Calib0834_thsq64
Rows: 122853
OBS_ID: 18092 -- 151486
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_2d', 'edisp_2d', 'events', 'gti', 'psf_3gauss', 'psf_table']


Observation table:
Observatory name: 'N/A'
Number of observations: 20485



In [36]:
src_pos = SkyCoord(359.94, -0.04, unit="deg", frame="galactic")

selection = dict(type='sky_circle', frame='galactic',
                 lon=src_pos.l ,
                 lat=src_pos.b,
                 radius=Angle(2, 'deg'),
                 border=Angle(0, 'deg'))

In [65]:
observations1 = data_store.obs_table.select_observations(selection)
obs_ids = list(observations1["OBS_ID"])
observations = data_store.get_observations(obs_ids)

In [66]:
obs2004 = observations.select_time(t2004)

In [67]:
len(obs2004)

0

# Analysis configuration

## Analyse 3D

In [27]:
conf_3d = AnalysisConfig()

# We define the datastore containing the data
conf_3d.observations.datastore = "$GAMMAPY_DATA/ash_stereo_Prod17_Calib0834_thsq64"

print(conf_3d)

AnalysisConfig

    general:
        log: {level: info, filename: null, filemode: null, format: null, datefmt: null}
        outdir: .
    observations:
        datastore: ../data/ash_stereo_Prod17_Calib0834_thsq64
        obs_ids: []
        obs_file: null
        obs_cone: {frame: null, lon: null, lat: null, radius: null}
        obs_time: {start: null, stop: null}
    datasets:
        type: 1d
        stack: true
        geom:
            wcs:
                skydir: {frame: null, lon: null, lat: null}
                binsize: 0.02 deg
                fov: {width: 5.0 deg, height: 5.0 deg}
                binsize_irf: 0.2 deg
            selection: {offset_max: 2.5 deg}
            axes:
                energy: {min: 0.1 TeV, max: 10.0 TeV, nbins: 30}
                energy_true: {min: 0.1 TeV, max: 10.0 TeV, nbins: 30}
        map_selection: [counts, exposure, background, psf, edisp]
        background: {method: reflected, exclusion: null}
        on_region: {frame: null, lon: nul

In [19]:
# We want a 3D analysis
conf_3d.datasets.type = "3d"

# We want to extract the data by observation and therefore to not stack them
conf_3d.datasets.stack = True

# Here is the WCS geometry of the Maps
conf_3d.datasets.geom.wcs.skydir = dict(
    frame="icrs", lon=359.94 * u.deg, lat=-0.04 * u.deg
)
conf_3d.datasets.geom.wcs.binsize = 0.02 * u.deg
conf_3d.datasets.geom.wcs.fov = dict(width=2 * u.deg, height=2 * u.deg)

# We define a value for the IRF Maps binsize
conf_3d.datasets.geom.wcs.binsize_irf = 0.2 * u.deg

# Define energy binning for the Maps
conf_3d.datasets.geom.axes.energy = dict(
    min=0.7 * u.TeV, max=10 * u.TeV, nbins=5
)
conf_3d.datasets.geom.axes.energy_true = dict(
    min=0.3 * u.TeV, max=20 * u.TeV, nbins=10
)

In [25]:
conf_3d.observations.obs_time.start = '2004-01-01T00:00:00'
conf_3d.observations.obs_time.stop = '2004-12-31T23:59:59'

In [28]:
analysis_3d = Analysis(conf_3d)
analysis_3d.get_observations()
analysis_3d.get_datasets()

Setting logging config: {'level': 'INFO', 'filename': None, 'filemode': None, 'format': None, 'datefmt': None}
Fetching observations.
Number of selected observations: 20485
Reducing spectrum datasets.


TypeError: The value must be a valid Python or Numpy numeric type.